In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
fnames = sorted(glob.glob('./data/*'))

In [3]:
df = pd.DataFrame(columns = ['BASIN' , 'CY' , 'YYYYMMDDHH' , 'TECHNUM' , 'TECH' , 'TAU' , 'LatN/S' , 'LonE/W' , 'VMAX' , 'MSLP' ,
'TY' , 'RAD' , 'WINDCODE' , 'RAD1' , 'RAD2' , 'RAD3' , 'RAD4' , 'RADP' , 'RRP' , 'MRD' , 'GUSTS' , 'EYE' ,
'SUBREGION' , 'MAXSEAS' , 'INITIALS' , 'DIR' , 'SPEED' , 'STORMNAME' , 'DEPTH' , 'SEAS' ,
'SEASCODE' , 'SEAS1' , 'SEAS2' , 'SEAS3' , 'SEAS4'])

In [4]:
for f in fnames:
    with open(f) as file:
        lines = [line.rstrip() for line in file]
    for x in lines:
        lst = x.split(', ')[0:35]
        new_row =  lst + [np.nan] * (35 - len(lst))
        df.loc[len(df)] = new_row

In [5]:
df['YYYYMMDDHH'] = pd.to_datetime(df['YYYYMMDDHH'], format = '%Y%m%d%H')
df.set_index('YYYYMMDDHH', inplace = True)
df.sort_index(inplace = True)

In [8]:
##  SELECTING ONLY ARABIAN SEA AND ONLY SOME OF THE AVAILABLE COLUMNS
df_select_columns = df[df['SUBREGION'] == '  A'][['CY', 'LatN/S', 'LonE/W', 'VMAX', 'MSLP', 'TY']]

df_select_columns.to_csv('CYCLONE_BEST_TRACK_AS.csv')